In [40]:
import pyttsx3
from ultralytics import YOLO
import cv2
import pandas as pd
from datetime import datetime
import os

# === AI action mappings ===
object_actions = {
    'laptop': "Would you like to work in your device?",
    'cell phone': "That is a smartphone right?",
    'person': "Do you need any assistance?",
    'light': "Should I turn on the smart light?",
    'cup': "Want me to check your water intake reminder?"
}

# === Initialize text-to-speech engine ===
engine = pyttsx3.init()

def chatbot_action(ai_action):
    # Speak the AI action using text-to-speech
    engine.say(ai_action)
    engine.runAndWait()

# === Excel file setup ===
excel_path = "detailed_object_log_with_actions.xlsx"
if not os.path.exists(excel_path):
    df_init = pd.DataFrame(columns=["Object", "Confidence", "Timestamp", "AI Action"])
    df_init.to_excel(excel_path, index=False)

# === Load YOLOv8 model ===
model = YOLO("yolov8n.pt")

# === Open webcam ===
cap = cv2.VideoCapture(0)

# Track objects already logged in current session
logged_objects = set()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]

    # Loop through detections
    for box in results.boxes:
        cls_id = int(box.cls[0].item())
        confidence = box.conf[0].item()
        label = model.names[cls_id]

        # Only log if confidence > 0.3 and not already logged
        if confidence > 0.3 and label not in logged_objects:
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            ai_action = object_actions.get(label, "")

            new_row = {
                "Object": label,
                "Confidence": round(confidence, 3),
                "Timestamp": timestamp,
                "AI Action": ai_action
            }

            # Read Excel, clean, and append
            df_existing = pd.read_excel(excel_path)
            df_existing.dropna(how='all', inplace=True)  # Remove empty rows
            df_updated = pd.concat([df_existing, pd.DataFrame([new_row])], ignore_index=True)
            df_updated.to_excel(excel_path, index=False)

            # Avoid logging same object again
            logged_objects.add(label)

            # Trigger chatbot action (speak AI action)
            if ai_action:  # Only speak if there's an action to trigger
                chatbot_action(ai_action)

    # Display live feed with annotations
    annotated_frame = results.plot()
    cv2.imshow("YOLOv8 Object Detection", annotated_frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 1 person, 217.7ms
Speed: 5.4ms preprocess, 217.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 178.5ms
Speed: 3.0ms preprocess, 178.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 107.4ms
Speed: 2.4ms preprocess, 107.4ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 couch, 105.1ms
Speed: 1.5ms preprocess, 105.1ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 102.0ms
Speed: 2.7ms preprocess, 102.0ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 98.5ms
Speed: 1.9ms preprocess, 98.5ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 couch, 94.3ms
Speed: 2.3ms preprocess, 94.3ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 90.8ms
Speed: 3.2ms preprocess, 90.8ms inference, 1.3ms postprocess 

In [42]:
import pandas as pd

# Read the Excel file properly using pandas
df = pd.read_excel("detailed_object_log_with_actions.xlsx")
print(df)

        Object  Confidence            Timestamp  \
0       person       0.886  2025-05-07 12:49:31   
1       laptop       0.659  2025-05-07 12:49:33   
2        donut       0.652  2025-05-07 12:49:37   
3       person       0.871  2025-05-07 13:04:21   
4   cell phone       0.309  2025-05-07 13:04:26   
5          cat       0.336  2025-05-07 13:04:30   
6       laptop       0.655  2025-05-07 13:04:32   
7        donut       0.415  2025-05-07 13:04:33   
8       banana       0.362  2025-05-07 13:04:39   
9       person       0.812  2025-05-07 13:16:01   
10       couch       0.333  2025-05-07 13:16:06   
11  cell phone       0.513  2025-05-07 13:16:08   
12         bed       0.412  2025-05-07 13:16:08   
13         cat       0.444  2025-05-07 13:16:15   
14      banana       0.598  2025-05-07 13:16:16   
15      laptop       0.393  2025-05-07 13:16:20   
16      toilet       0.312  2025-05-07 13:16:25   
17       clock       0.328  2025-05-07 13:16:26   

                              